In [ ]:
from tensorflow import data

In [ ]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
AUTOTUNE = data.experimental.AUTOTUNE

In [ ]:
import pathlib
import keras

In [ ]:
data_dir = keras.utils.get_file(origin='http://download.tensorflow.org/example_images/flower_photos.tgz', fname='flower_photos',
                                untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [ ]:
class_names = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
class_names

In [ ]:
roses = list(data_dir.glob('roses/*'))

In [ ]:
for image_path in roses[:3]:
    display.display(Image.open(str(image_path)))

In [ ]:
#Keras.preprocessing to load images
#The 1./255 is to convert from uint8 to float32 in range [0,1]
image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
BATCH_SIZE=32
IMG_HEIGHT=224
IMG_WIDHT =224
STEPS_PER_EPOCH =np.ceil(image_count/BATCH_SIZE)

In [ ]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    target_size=(IMG_HEIGHT,IMG_WIDHT),
                                                    classes = list(class_names))

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(class_names[label_batch[n]==1][0].title())
        plt.axis('off')
        

In [ ]:
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

In [ ]:
list_ds = data.Dataset.list_files(str(data_dir/'*/*'))

In [ ]:
for f in list_ds.take(5):
    print(f.numpy())

In [ ]:
import tensorflow as tf

In [ ]:
def get_label(file_path):
    #convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    #the second to last is class-directory
    return parts[-2] == class_names

In [ ]:
def decode_img(img):
    #convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    #Use "convert_image_dtype" to convert to floats in the [0,1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    #resize the image to the desired size
    return tf.image.resize(img,[IMG_HEIGHT,IMG_WIDHT])

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    #load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label 

In [ ]:
#Set 'num_parallel_calls' so multiple images are loaded/processed in parallel
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds.take(1):
    print("Image shape:",image.numpy().shape)
    print("Label:", label.numpy())

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    #This is a small dataset, only load it once, and keep it in memory.
    #use '.cache(filename)' to cache preprocessing work for datasets that don't fit in memory
    if cache:
        if isinstance(cache,str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
            
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    
    #Repeat forever
    ds = ds.repeat()
    
    ds = ds.batch(BATCH_SIZE)
    
    #'prefetch' lets the dataset fetch bacthes in the background while the model is training
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

In [ ]:
train_ds = prepare_for_training(labeled_ds)

In [ ]:
image_batch, label_batch = next(iter(train_ds))

In [ ]:
show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
import time
default_timeit_steps = 1000

In [ ]:
def timeit(ds, steps=default_timeit_steps):
    start = time.time()
    it = iter(ds)
    for i in range(steps):
        batch = next(it)
        if i%10 == 0:
            print('.',end='')
    print()
    end = time.time()
    
    duration = end-start
    print("{} batches: {} s".format(steps,duration))
    print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))    

In [ ]:
#keras preprocessing
timeit(train_data_gen)

In [ ]:
#tf.data
timeit(train_ds)

In [ ]:
uncached_ds = prepare_for_training(labeled_ds,cache=False)
timeit(uncached_ds)

In [ ]:
filecached_ds = prepare_for_training(labeled_ds, cache = './flowers.tfcache')
timeit(filecached_ds)